In [ ]:
import ee
import geemap.core as geemap

In [ ]:
ee.Authenticate()

True

In [ ]:
ee.Initialize(project = "shining-glyph-466811-u6")

In [ ]:
COUNTRY = 'Morocco'
START_YEAR = 2010
END_YEAR   = 2024       # inclusive
SCALE_M    = 1000       # ~1km for MODIS

adm1 = (ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level1') #pour les regions
        .filter(ee.Filter.eq('ADM0_NAME', COUNTRY))
        .select(['ADM0_NAME','ADM1_NAME']))

In [ ]:
# Datasets
ic_ndvi   = ee.ImageCollection('MODIS/061/MOD13A2').select('NDVI')
ic_chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')
ic_era5m  = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select('temperature_2m')
ic_co = (ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density'))
ic_viirs = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').select('avg_rad')

In [ ]:
years  = ee.List.sequence(START_YEAR, END_YEAR)
months = ee.List.sequence(1, 12)

def masked_band(name):
    """Return a fully masked 1-band image."""
    return ee.Image.constant(0).updateMask(ee.Image.constant(0)).rename(name)

def monthly_panel(y, m):
    y = ee.Number(y); m = ee.Number(m)
    start = ee.Date.fromYMD(y, m, 1)
    end   = start.advance(1, 'month')

    # NDVI monthly mean → scale to [-1, 1]
    ndvi_col = ic_ndvi.filterDate(start, end)
    ndvi = ee.Image(ee.Algorithms.If(
        ndvi_col.size().gt(0),
        ndvi_col.mean().multiply(0.0001).rename('ndvi'),
        masked_band('ndvi')
    ))

    # CHIRPS monthly total (sum of daily mm)
    chirps_col = ic_chirps.filterDate(start, end)
    precip = ee.Image(ee.Algorithms.If(
        chirps_col.size().gt(0),
        chirps_col.sum().rename('precip_mm'),
        masked_band('precip_mm')
    ))

    # ERA5-Land monthly mean 2m temperature → °C
    era5_col = ic_era5m.filterDate(start, end)
    t2m = ee.Image(ee.Algorithms.If(
        era5_col.size().gt(0),
        era5_col.mean().subtract(273.15).rename('t2m_c'),
        masked_band('t2m_c')
    ))
    co_col = ic_co.filterDate(start, end)
    co = ee.Image(ee.Algorithms.If(
        co_col.size().gt(0),
        co_col.mean().rename('co_col_molm2'),
        masked_band('co_col_molm2')
    ))
      # --- VIIRS lights monthly mean (nW·sr⁻¹·cm⁻²) ---
    viirs_col = ic_viirs.filterDate(start, end)
    viirs = ee.Image(ee.Algorithms.If(
        viirs_col.size().gt(0),
        viirs_col.mean().rename('viirs_rad'),
        masked_band('viirs_rad')
    ))

    cube = ee.Image.cat([ndvi, precip, t2m, co, viirs])

    stats = (cube.reduceRegions(
                collection=adm1,
                reducer=ee.Reducer.mean(),
                scale=SCALE_M,
                tileScale=4
             )
             .map(lambda f: f.set({'year': y, 'month': m})))
    return stats


In [ ]:

# BUILD COMPLETE FEATURE COLLECTION (robust flatten)
# -------------------------------
years = ee.List.sequence(START_YEAR, END_YEAR)
months = ee.List.sequence(1, 12)

# Make a flat list of {year, month} dictionaries
ym_list = years.map(lambda y: months.map(lambda m: ee.Dictionary({'y': y, 'm': m}))).flatten()

# Map each {y,m} to a FeatureCollection, then FLATTEN
all_stats = ee.FeatureCollection(
    ym_list.map(lambda d: monthly_panel(ee.Dictionary(d).get('y'),
                                       ee.Dictionary(d).get('m')))
).flatten()

# -------------------------------
# EXPORT TO GOOGLE DRIVE AS CSV
# -------------------------------
task = ee.batch.Export.table.toDrive(
    collection = all_stats,
    description = f"{COUNTRY}_NDVI_Rain_Temp_2010_2024",
    folder = 'EarthEngineExports',            # Drive folder name (will be created if needed)
    fileNamePrefix = f"{COUNTRY}_NDVI_Rain_Temp_2010_2024",
    fileFormat = 'CSV',
    selectors = ['ADM0_NAME','ADM1_NAME','year','month','ndvi','precip_mm','t2m_c', 'co_col_molm2', 'viirs_rad']  # column order
)
task.start()
import time
while True:
    s = task.status()
    print(s.get('state'), s.get('error_message'))
    if s.get('state') in ('COMPLETED', 'FAILED', 'CANCELLED'):
        break
    time.sleep(10)
print(task.status())
print(" Export started! Check Google Drive > EarthEngineExports")



READY None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
RUNNING None
COMPLETED None
{'state': 'COMPLETED', 'description': 'Morocco_NDVI_Rain_Temp_2010_2024', 'priority': 100, 'creation_timestamp_ms': 1762386373073, 'update_timestamp_ms': 1762386472401, 'start_timestamp_ms': 1762386381043, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1828X2EajL05zGo-tads_Wu09aLWwbhTt'], 'attempt': 1, 'batch_eecu_usage_seconds': 7066.3369140625, 'id': 'MAQXLEA5Q2LFOJSYAVS32BNY', 'name': 'projects/shining-glyph-466811-u6/operations/MAQXLEA5Q2LFOJSYAVS32BNY'}
 Export started! Check Google Drive > EarthEngineExports
